In [ ]:
!pip install -U langchain-community

In [5]:
!pip install beautifulsoup4


  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


You should consider upgrading via the 'd:\langchain\textsummarizer\summariser\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
from langchain.document_loaders import WebBaseLoader

urls = [
    'https://en.wikipedia.org/wiki/Ratan_Tata',
    'https://en.wikipedia.org/wiki/elonmusk',
    'https://en.wikipedia.org/wiki/JRD_Tata'
]

documents = []
for url in urls:
    loader = WebBaseLoader(url)
    docs = loader.load()  # Load documents from the URL
    documents.extend(docs)  # Append the loaded docs to the main document list

# Now `documents` contains all the documents from the specified URLs.


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
documents[2].metadata['source']

'https://en.wikipedia.org/wiki/JRD_Tata'

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
chunked_docs=splitter.split_documents(documents)
for chunk in chunked_docs:
    print(len(chunk.page_content))

142
192
155
177
181
175
181
181
179
170
137
196
176
149
37
51
181
191
195
196
109
191
192
81
192
198
198
29
167
50
195
195
199
128
194
193
194
68
18
192
152
11
191
196
191
194
199
133
47
199
197
175
196
199
28
191
192
199
194
194
179
197
195
190
196
94
12
189
31
29
196
197
97
188
52
198
193
189
196
194
199
78
36
199
190
194
43
196
194
199
53
38
198
197
35
123
193
88
30
197
150
40
189
87
18
197
142
60
198
60
198
136
197
192
29
193
199
193
113
191
181
197
196
127
195
116
124
193
194
44
21
195
191
139
192
184
163
185
189
197
179
198
198
187
176
197
174
185
174
18
181
141
194
16
144
163
141
130
89
136
193
157
135
180
171
139
173
120
174
115
167
134
196
30
164
198
58
130
134
194
67
163
177
196
16
113
158
177
192
46
178
170
181
147
165
156
153
175
191
172
194
115
135
154
171
193
162
138
180
140
101
150
188
177
141
153
156
145
174
192
45
100
169
198
32
196
191
33
170
118
161
138
105
105
107
126
114
164
153
181
166
51
152
154
183
184
198
32
184
152
164
113
146
158
182
124
113
162
50
171
172
15

In [20]:
!pip install sentence-transformers

You should consider upgrading via the 'd:\langchain\textsummarizer\summariser\scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)


C:\Users\USER\AppData\Local\Temp\ipykernel_16776\2267500551.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
D:\Langchain\TextSummarizer\summariser\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
doc_texts = [chunk.page_content for chunk in chunked_docs]  # List of chunks
doc_embeddings = embeddings.embed_documents(doc_texts)


In [29]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.9.0-cp39-cp39-win_amd64.whl (14.9 MB)


You should consider upgrading via the 'd:\langchain\textsummarizer\summariser\scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_texts(texts=doc_texts, embedding=embeddings)
vectorstore.save_local("faiss_index")  # "faiss_index" is the folder where the index will be saved

print("FAISS vector store saved successfully!")

FAISS vector store saved successfully!


In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})  # Retrieve top 4 relevant chunks

In [8]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    temperature=0.7,
    groq_api_key="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",
    model_name="llama-3.1-70b-versatile"
)

In [9]:
response=llm.invoke("Name of Ratan Tata?")
print(response.content)

The full name of Ratan Tata is Ratan Naval Tata.


In [10]:
query = "who is elon musk?"
retrieved_docs = vectorstore.similarity_search(query, k=4)  # Retrieve to

In [11]:
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i + 1}: {doc.page_content}")
combined_context = "\n".join([doc.page_content for doc in retrieved_docs])


Document 1: Wikipedia does not have an article with this exact name. Please search for Elonmusk in Wikipedia to check for alternative titles or spellings.
Document 2: Search for "Elonmusk" in existing articles.
Look for pages within Wikipedia that link to this title.
Document 3: George Fernandes
P. B. Gajendragadkar
Benjamin Gilman
Ismaïl Omar Guelleh
Zakir Husain
V. R. Krishna Iyer
Jagmohan
Lakshmi Chand Jain
Arun Jaitley
Aditya Nath Jha
Murli Manohar Joshi
Anerood Jugnauth
Document 4: George Fernandes
P. B. Gajendragadkar
Benjamin Gilman
Ismaïl Omar Guelleh
Zakir Husain
V. R. Krishna Iyer
Jagmohan
Lakshmi Chand Jain
Arun Jaitley
Aditya Nath Jha
Murli Manohar Joshi
Anerood Jugnauth


In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

prompt_template = """
Given the following context:
{context}

Answer the following question:
{question}
"""

# Step 4: Create the PromptTemplate and LLMChain
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# Step 5: Run the LLMChain with the combined context and query
result = llm_chain.run({
    "context": combined_context,
    "question": query
})

# Step 6: Output the final answer
print("Answer:", result)

C:\Users\USER\AppData\Local\Temp\ipykernel_16776\3708327967.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\USER\AppData\Local\Temp\ipykernel_16776\3708327967.py:24: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({


Answer: Based on the given context, I couldn't find any information about Elon Musk from the provided text. However, I can provide general information about Elon Musk.

Elon Musk is a business magnate, industrial designer, and engineer. He is the CEO and CTO of SpaceX, CEO and product architect of Tesla, Inc., founder of Neuralink and The Boring Company, and co-founder of PayPal.

To find more information about Elon Musk, I would recommend searching for his name in Wikipedia, where you can find a detailed article about his life, career, and achievements.
